In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

In [ ]:
df = pd.read_csv('/kaggle/input/loan-approval-classification-data/loan_data.csv')

for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.lower()

In [ ]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
categorical = list(df.dtypes[df.dtypes == 'object'].index)

numerical = ['person_age', 'person_income', 'person_emp_exp',
             'loan_amnt', 'loan_int_rate', 'loan_percent_income', 
             'cb_person_cred_hist_length','credit_score']

In [ ]:
def train(df_train, y_train):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model= XGBClassifier(
        max_depth=9, n_estimators=149, 
        learning_rate=0.06701465630034122, 
        subsample=0.6531573257784621,
        abs=False, eval_metric='mlogloss')
    
    model.fit(X_train, y_train)
    
    return dv, model

In [ ]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_prob = model.predict_proba(X)[:, 1]
    y_pred = model.predict(X)

    return y_prob, y_pred

In [ ]:
dv, model = train(df_full_train, df_full_train.loan_status.values)
y_prob, y_pred = predict(df_test, dv, model)

y_test = df_test.loan_status.values
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print(f'AUC: {auc}, Accuracy: {accuracy}')

In [ ]:
#SAVE THE MODEL

In [ ]:
import pickle

with open('model.bin', 'wb') as f_out:
    pickle.dump((dv, model), f_out)

In [ ]:
# Load the modelimport pickle

with open('model.bin', 'rb') as f_in: 
    dv, model = pickle.load(f_in)

In [ ]:
dv

In [ ]:
model

In [ ]:
applicant = {'person_gender': 'male',
     'person_education': 'master',
     'person_home_ownership': 'own',
     'loan_intent': 'personal',
     'previous_loan_defaults_on_file': 'yes',
     'person_age': 25.0,
     'person_income': 85320.0,
     'person_emp_exp': 2,
     'loan_amnt': 4000.0,
     'loan_int_rate': 7.42,
     'loan_percent_income': 0.05,
     'cb_person_cred_hist_length': 2.0,
     'credit_score': 709}

In [ ]:
X = dv.transform([applicant])

In [ ]:
y_prob = model.predict_proba(X)[0, 1]

In [ ]:
print('input:', applicant)
print('output:', y_prob)